In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='INFO')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

The CD4069UB device consist of six CMOS inverter circuits. These devices are intended for all generalpurpose inverter applications where the mediumpower TTL-drive and logic-level-conversion. This device is not ideal for linear amplification, but can be used as such and will add a lot of soft-clipping. Soft-clipping is expected from a valve. This makes this device special for musical usage and can be found in guitar distortion pedals [[2](http://www.runoffgroove.com/ubescreamer.html)] or the wasp filter [[3](https://www.schmitzbits.de/wasp.html)]. 
{: class="mt-6"}

the chip should be powered by +5V/GND. With higher voltage the heat dissipation will be to big and the chip will be damaged. the chip is powered on pin 7 and 14 with +5V and Ground. the other pins can be used as amplifier. 

# sawtooth oszillator

the simplest circuit to begin with is a voltage follower or buffer. the buffer confifuration looks similar to the opamp buffer. but there are some differences. the first important difference is that the feedback will also do the biasing of the input signal. when an ac signal is applied to the buffer the output will be a dc signal, as long as the chip is able to provide the needed biasing feedback. the gain can be calculated similar to the opamp configuration. but there will be less gain as we would excpect from the calculation. 

In [ ]:
d = schemdraw.Drawing(unit=2, inches_per_unit=0.6, lw=1.8)

d += ( U1 := elm.Opamp() )
d += ( L1 := elm.Line().left().length(d.unit/4).at(U1.in1) )
d += elm.Dot()
d += ( R1 := elm.Resistor().label("R4\n270k"))
d += elm.Tag().label("cv")
d += elm.Line().left().at(U1.in2).length(d.unit/4)
d += elm.Ground()
     
#Feedback
d += ( L2 := elm.Line().up().at(L1.end) )
d += elm.Dot()
d += elm.Capacitor().right().length(d.unit*1.5).label("C1\n1n")
d += elm.Dot()
d += ( L3 := elm.Line().down().toy(U1.out) )
d += elm.Dot()

#feedback transistor
d += elm.Line().up().at(L2.end)
d += elm.Line().right().length(d.unit*0.4)
d += ( Q1 := elm.BjtPnp(circle=True).anchor("collector").theta(-90) )
d += elm.Line().right().tox(L3.end)
d += elm.Line().down()

d += elm.Line().up().length(d.unit/4).at(Q1.base)
d += elm.Line().left().length(d.unit/2)
d += elm.Resistor().label("R3\n22k")
d += elm.Line().tox(R1.end)

# cv input amplifier 
d += ( Uin := elm.Opamp().right().anchor('out') )
d += elm.Line().left().length(d.unit/4).at(Uin.in2)
d += elm.Ground()
d += ( Lin :=  elm.Line().left().length(d.unit/4).at(Uin.in1) )
d += elm.Dot()
d += elm.Resistor().label('R1\n100k\n')
d += elm.Tag().label("pulse")

#feedback input amplifier
d += elm.Line().up().at(Lin.end)
d += elm.Resistor().right().length(d.unit*1.5).label('R2\n100k')
d += elm.Line().down().toy(Uin.out)
d += elm.Dot()

#output amplifier
d += elm.Line().right().at(U1.out).length(d.unit/2)
d += ( R5 := elm.Resistor().label('R5\n10k'))
d += ( Lout := elm.Line().right().length(d.unit/4) )
d += elm.Dot()
d += elm.Line().right().length(d.unit/4)
d += ( U3 := elm.Opamp().anchor('in2'))
d += elm.Line().down().at(Lout.end)
d += elm.Line().left().length(d.unit/4)
d += elm.Resistor().left().label('R6\n120k').label('+15V', loc='left')
#d += elm.Vdd().label('+15V')

#feedback output amplifier
d += elm.Line().left().at(U3.in1).length(d.unit/4)
d += ( Lref := elm.Line().up())
d += elm.Dot()
d += elm.Resistor().right().label('R8\n100k').length(d.unit*1.5)
d += elm.Line().down().toy(U3.out)
d += elm.Dot()

d += elm.Resistor().left().at(Lref.end).label('R7\n33k')
d += elm.Ground()

d += elm.Line().right().at(U3.out).length(d.unit/2)
d += elm.Tag().label("OUT")


d.draw()

This is the first setup with the 4069 as voltage follower. C1 and C3 are the dc blocking capacitors. When we choose R1 and R2 as 100kOhm we would excpect a gain of one. 

In [ ]:
circuit = Circuit('cmos buffer')
circuit.include(spice_library['TL072'])
circuit.include(spice_library['BC556B'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'vCharge', circuit.gnd, 'DC 0.594000V')
circuit.V('5', 'vPulse', circuit.gnd, 'DC 0 PULSE ( 0 5V 0 0 0 0.001ms 1.1363636363636ms )')

#pulse amplifier
circuit.R(1, 'vPulse', 'R1', 100@u_kΩ)
circuit.R(2, 'R1', 'xPulse', 100@u_kΩ)
circuit.X(1, 'TL072', circuit.gnd, 'R1', '+15V', '-15V', 'xPulse')

#integrator
circuit.R(4, 'vCharge', 'R4', 270@u_kΩ)
circuit.X(2, 'TL072', circuit.gnd, 'R4', '+15V', '-15V', 'OUTi')
circuit.C(1, 'R4', 'OUTi', 1@u_nF)

circuit.R(3, 'xPulse', 'R3', 22@u_kΩ)
circuit.BJT(1, 'R4', 'R3', 'OUTi', model="BC556B")

#output buffer
circuit.R(5, 'OUTi', 'R5', 10@u_kΩ)
circuit.R(6, 'R5', '+15V', 120@u_kΩ)

circuit.X(3, 'TL072', 'R5', 'X3', '+15V', '-15V', 'OUT')
circuit.R(8, 'X3', 'OUT', 100@u_kΩ)
circuit.R(7, 'X3', circuit.gnd, 33@u_kΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.initial_condition(OUTi=0.0)
analysis  = simulator.transient(step_time=1@u_us, start_time=1@u_ms, end_time=3@u_ms)

fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
buffer_axis, = ax1_buffer.plot(u_ms(analysis['OUT'].abscissa), analysis['OUT'], color='Red')
buffer_axis, = ax1_buffer.plot(u_ms(analysis['OUTi'].abscissa), analysis['OUTi'], color='Blue')
ax1_buffer.legend(('Vout [V]', 'Vintegtator'), loc=(0.7,0.8))

plt.tight_layout()
plt.show()


The blue line is the input signal (5V p2p) and the red the output signal with 100kOhm resistors. The output is a little less then excpected. For the other signals the resistor R2 is replaced with 125kΩ, 150kΩ, 175kΩ and 200kΩ. With 150kΩ the gain is roughly one. We also see that the ouput is not symetrical. The clipping occures earlier for the negative part of the signal. With higher gain the signal is soft clipped. 

In [ ]:
dac = 2.048 * 1 * (2048 / ( 2 ** 12) ) 
print( dac )

In [ ]:
dac = 1.024 / (2048 / ( 2 ** 12) ) 
print( dac )

In [ ]:
notes = [ 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B', 
    'C', 'C#/Db ', 'D', 'D#/Eb ', 'E', 'F', 'F#/Gb ', 'G', 'G#/Ab', 'A', 'A#/Bb ', 'B'
]

octaves = [
    '-5', '-5 ', '-5', '-5 ', '-5', '-5', '-5 ', '-5', '-5', '-5', '-5 ', '-5', 
    '-4', '-4 ', '-4', '-4 ', '-4', '-4', '-4 ', '-4', '-4', '-4', '-4 ', '-4', 
    '-3', '-3 ', '-3', '-3 ', '-3', '-3', '-3 ', '-3', '-3', '-3', '-3 ', '-3', 
    '-2', '-2 ', '-2', '-2 ', '-2', '-2', '-2 ', '-2', '-2', '-2', '-2 ', '-2', 
    '-1', '-1 ', '-1', '-1 ', '-1', '-1', '-1 ', '-1', '-1', '-1', '-1 ', '-1', 
    '0', '0 ', '0', '0 ', '0', '0', '0 ', '0', '0', '0', '0 ', '0', 
    '1', '1 ', '1', '1 ', '1', '1', '1 ', '1', '1', '1', '1 ', '1', 
    '2', '2 ', '2', '2 ', '2', '2', '2 ', '2', '2', '2', '2 ', '2', 
    '3', '3 ', '3', '3 ', '3', '3', '3', '3', '3', '3', '3 ', '3', 
    '4', '4 ', '4', '4 ', '4', '4', '4 ', '4', '4', '4', '4 ', '4', 
    '5', '5 ', '5', '5 ', '5', '5', '5 ', '5', '5', '5', '5 ', '5'
]

In [ ]:
_dac_v_low = 2.048
_dac_v_high =  4.096

def calculate_dac(voltage) :
    result = []
    if voltage < _dac_v_low :
        result.append(0)
        result.append(0)
        result.append(0) 
        dac1_value = voltage / (_dac_v_low / 4095)
        result.append(dac1_value)
        result.append(0)
        result.append((_dac_v_low / 4095 * dac1_value))

    elif voltage <= _dac_v_high :
        result.append(0)
        result.append(_dac_v_low)
        result.append(0) 
        dac1_value = (voltage - _dac_v_low) / (_dac_v_low / 4095)
        result.append(4095)
        result.append(_dac_v_low)
        result.append((_dac_v_low / 4095 * dac1_value))

    elif voltage <= _dac_v_high + _dac_v_low :
        result.append(1)
        result.append(_dac_v_high)
        result.append(0) 
        dac1_value = (voltage - _dac_v_high) / (_dac_v_low / 4095)
        result.append(4095)
        result.append(_dac_v_high)
        result.append((_dac_v_low / 4095 * dac1_value))

    else :
        result.append(1)
        result.append(_dac_v_high)
        result.append(1) 
        dac1_value = (voltage - _dac_v_high) / (_dac_v_high / 4095)
        result.append(4095)
        result.append(_dac_v_high)
        result.append((_dac_v_high / 4095 * dac1_value))
        
    return result


In [ ]:
a = 440
v_step = 1 / 12

table = '''
<div class="section">
<div class="container">
<div class="content">
<table class="table">
  <thead>
    <tr>
      <th>Midi Note</th>
      <th>Octave></th>
      <th>Note</th>
      <th>Frequency [Hz]</th>
      <th>CV [V]</th>
      <th>time [ms]</th>
      <th>Scaling</th>
      <th>Final Frequency [Hz]</th>
      <th>Deviation [%]</th>
      <th>DAC [V]</th>
      <th> DAC1 Gain </th>
      <th> DAC2 Gain </th>
      <th> DAC1 value</th>
      <th> DAC2 value </th>
      <th> DAC1 [V] </th>
      <th> DAC2 [V] </th>
      <th> DAC1 real [V] </th>
      <th> DAC2 real </th>
    </tr>
  </thead>
  <tbody>
'''

cpu_freq = 48000000
prescaler = 48 * 2
timer_freq = cpu_freq / prescaler
R = 270@u_kΩ
C = 1@u_nF
#V = 4.422@u_V
V = 2.5@u_V

for s in np.arange(0, 128) :
    
    _freq = a * 2 ** ((s-69) / 12 )
    _time = 1 / _freq
    _scaling = timer_freq / _freq
    _round_scaling = int(_scaling)
    _real_freq = timer_freq/_round_scaling
    _deviation = _freq / _real_freq * 100
    #voltage = (float(val)/4096) * 3.3
    voltage = R * C * V / _time
        
    dac = voltage / (5/4095) 

    table += '<tr>'
    table += '<td>  %s  </td>' % s
    table += '<td>  %s </td>' % octaves[s]
    table += '<td> %s </td>' % notes[s]
    table += '<td> %.2f </td>' % _freq
    table += '<td> %.2f </td>' % ((s-69) * v_step)
    table += '<td> %.4f </td>' % (_time*1000)
    table += '<td> %d (%.2f) </td>' % ( _round_scaling, _scaling )
    table += '<td> %.2f </td>' % _real_freq
    table += '<td> %.4f </td>' % (100-_deviation)
    table += '<td> %f </td>' % voltage
    
    result = calculate_dac(voltage)
    table += '<td>%d </td>' % result[0]
    table += '<td>%d </td>' % result[2]
    table += '<td> %d </td>'  % result[1]
    table += '<td> %d </td>' % result[3]
    table += '<td> %.4f </td>' % result[4]
    table += '<td> %.4f </td>' % result[5]
    table += '<td> %.4f </td>' % (result[4] + result[5])
    table += '<td> %.2f </td>' % ( (result[4] + result[5]) / voltage * 100)
    table += '</tr>'

table += "</tbody></table></div></div></div>"

from IPython.display import display, HTML
display(HTML(table))

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<div class="hero is-medium" style="background: url('{{ '/assets/cmos_buffer_files/tmb_ube_screamer.jpg' | relative_url }}') no-repeat center center; background-size: cover; background-attachment: fixed;">
   <div class="hero-body">
        <div class="content has-text-centered">
        </div>
    </div>
</div>
'''))

In [ ]:
for i in np.arange( 0, 9, 1) :
    print( -33000 * ( (i/100000 ) + (-10/100000) ) )

In [ ]:
print( 3.3 / 4096)
print( 0.0833 / 3)

In [ ]:
old_val = 0
for i in np.arange( 0, 0.9167, 0.0833 ) :
    val = -33000 * ( (i/100000 ) + (-10/100000) );
    print( "CV: %f -> %f " % (val, val-old_val) )
    old_val = val
                   

In [ ]:
circuit = Circuit('cv buffer')
circuit.include(spice_library['TL072'])

circuit.V('1', '+3.3V', circuit.gnd, 'DC 3.3')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'vCharge', circuit.gnd, 'DC 0.594000V')
circuit.V('5', 'vPulse', circuit.gnd, 'DC 0 PULSE ( 0 5V 0 0 0 0.001ms 1.1363636363636ms )')

#pulse amplifier
circuit.R(1, 'vPulse', 'R1', 100@u_kΩ)
circuit.R(2, 'R1', 'xPulse', 100@u_kΩ)
circuit.X(1, 'TL072', circuit.gnd, 'R1', '+15V', '-15V', 'xPulse')


# references

1. [thea.codes](https://blog.thea.codes/the-design-of-the-juno-dco/) The Design of the Roland Juno oscillators.
1. [microchip.com](https://ww1.microchip.com/downloads/en/DeviceDoc/20002249B.pdf) MCP4802/4812/4822 datasheet.
1. [NGF-E Waveshaper](https://www.haraldswerk.de/NGF_E/NGF_E_Shape/NGF_E_Shape.html)